<a href="https://colab.research.google.com/github/NemaVatsala/101trial/blob/master/HPC_Workshop_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls /usr/local/cuda

bin	EULA.txt  lib64		    nvml     Sanitizer	targets
compat	extras	  libnvvp	    nvvm     share	tools
doc	include   nsightee_plugins  samples  src	version.txt


In [ ]:
%%writefile numdevices.c

#include <stdio.h>
#include <omp.h>

int main() {
    int Numdevices = omp_get_num_devices();
    printf("number of devices= %d", Numdevices);
}

Writing numdevices.c


In [ ]:
!true | add-apt-repository ppa:ubuntu-toolchain-r/test

 Toolchain test builds; see https://wiki.ubuntu.com/ToolChain

 More info: https://launchpad.net/~ubuntu-toolchain-r/+archive/ubuntu/test
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [348 kB]
Hit:

In [ ]:
!apt install gcc-10 g++-10 gcc-10-offload-nvptx libgomp1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cpp-10 gcc-10-base libasan6 libatomic1 libcc1-0 libgcc-10-dev libgcc-s1
  libgomp-plugin-nvptx1 libitm1 liblsan0 libquadmath0 libstdc++-10-dev
  libstdc++6 libtsan0 libubsan1 nvptx-tools
Suggested packages:
  gcc-10-locales g++-10-multilib gcc-10-doc libstdc++6-10-dbg gcc-10-multilib
  libgcc-s1-dbg libgomp1-dbg libitm1-dbg libatomic1-dbg libasan6-dbg
  liblsan0-dbg libtsan0-dbg libubsan1-dbg libquadmath0-dbg libstdc++-10-doc
  nvidia-cuda-toolkit
The following NEW packages will be installed:
  cpp-10 g++-10 gcc-10 gcc-10-base gcc-10-offload-nvptx libasan6 libgcc-10-dev
  libgcc-s1 libgomp-plugin-nvptx1 libstdc++-10-dev libubsan1 nvptx-tools
The following packages will be upgraded:
  libatomic1 libcc1-0 libgomp1 libitm1 liblsan0 libquadmath0 libstdc++6
  libtsan0
8 upgraded, 12 newly installed, 0 to remove and 63 not upgraded.
Need to ge

In [ ]:
!ls -l /usr/local/cuda

lrwxrwxrwx 1 root root 9 Mar 18 13:30 /usr/local/cuda -> cuda-11.0


In [ ]:
!ln -sfnv /usr/local/cuda-10.0/ /usr/local/cuda

'/usr/local/cuda' -> '/usr/local/cuda-10.0/'


In [ ]:
%%writefile first.c

#include <omp.h>
#include <stdio.h>
#include <stdlib.h>

#define N 1024

int A[N][N], B[N][N], C[N][N];

int main() {
  srand(3);

  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      A[i][j] = rand() % N + 1;
      B[i][j] = rand() % N + 1;
    }
  }

printf("Total Devices = %d \n", omp_get_num_devices());
#pragma omp target data map(to : A, B) map(from : C)
  {
#pragma omp parallel for
    for (int i = 0; i < N; i++) {
      for (int j = 0; j < N; j++)
        C[i][j] = A[i][j] * B[i][j];
    }
  }

 return 0;
}

Writing first.c


In [ ]:
!g++-10 -fno-lto -fopenmp -foffload=nvptx-none -fstack-protector first.c -o first

In [ ]:
!nvprof ./first

==1746== NVPROF is profiling process 1746, command: ./first
Total Devices = 1 
==1746== Profiling application: ./first
==1746== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   74.85%  1.8347ms         2  917.37us  912.80us  921.95us  [CUDA memcpy HtoD]
                   25.15%  616.36us         1  616.36us  616.36us  616.36us  [CUDA memcpy DtoH]
      API calls:   82.76%  279.96ms         1  279.96ms  279.96ms  279.96ms  cuCtxCreate
                   15.95%  53.968ms         1  53.968ms  53.968ms  53.968ms  cuCtxDestroy
                    0.73%  2.4783ms         2  1.2392ms  1.2347ms  1.2436ms  cuMemcpyHtoD
                    0.29%  971.55us         1  971.55us  971.55us  971.55us  cuMemcpyDtoH
                    0.10%  354.62us         1  354.62us  354.62us  354.62us  cuDeviceGetPCIBusId
                    0.07%  233.11us         1  233.11us  233.11us  233.11us  cuMemAlloc
                    0.05%  160.80us 

In [ ]:
%%writefile program1.c
#include<stdio.h>
#include<omp.h>
int main ( ) {
    #pragma omp parallel
    {
        printf("Hello World!\n")
    }
    return 0;
  
}

Overwriting program1.c


In [ ]:
!gcc program1.c -o serial


program1.c: In function ‘main’:
program1.c:7:5: error: expected ‘;’ before ‘}’ token
     }
     ^


In [ ]:
./a.out

SyntaxError: ignored